In [1]:
# Set Up:
import sys
USING_COLAB = 'google.colab' in sys.modules

if USING_COLAB:
    !apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1
    !pip install -U renderlab
    !pip install -U colabgymrender
    !pip install -U moviepy==0.2.3.5
    !pip install imageio==2.4.1
    !pip install --upgrade AutoROM
    !AutoROM --accept-license
    !pip install stable_baselines3  
    !pip install "gymnasium[atari, accept-rom-licesnse]"

import numpy as np
import ale_py
import shimmy
import gymnasium as gym
import renderlab as rl
import random
import matplotlib.pyplot as plt
from colabgymrender.recorder import Recorder
from copy import deepcopy

from torch.utils.data import DataLoader
from torch import nn
import torch

from tqdm import tqdm, trange

seed = 24
data_seed = 700



In [2]:
from stable_baselines3 import PPO
from stable_baselines3.common.callbacks import EvalCallback
from stable_baselines3.common.vec_env.base_vec_env import VecEnv
from stable_baselines3.common.env_checker import check_env # for regular envs
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.env_util import make_atari_env
from stable_baselines3.common.vec_env import DummyVecEnv, SubprocVecEnv
from stable_baselines3.common.vec_env import VecFrameStack
# not using make_atari_env bc that wraps it in an atari wrapper only for v4 stuff (needs verification)

In [3]:

print('gym:', gym.__version__)
print('ale_py:', ale_py.__version__)
print("GPU is available:", torch.cuda.is_available())

gym: 0.29.1
ale_py: 0.8.1
GPU is available: True


In [4]:
print(torch.version.cuda)
print(torch.__version__)

11.8
2.2.2


# Functions from A5 starter code

In [5]:
  # Setting the seed to ensure reproducability
def reseed(seed, env=None):
    '''
        Sets the seed for reproducibility

        When @param env is provided, also sets the
        random number generataor of the gym environment
        to this particular seed
    '''
    torch.manual_seed(seed)
    random.seed(seed)
    np.random.seed(seed)

    if env is not None:
        env.unwrapped._np_random = gym.utils.seeding.np_random(seed)[0]

reseed(seed)

In [6]:
def visualize(env_name='ALE/BasicMath-v5', algorithm=None, video_name="test", env_args={}):
    """Visualize a policy network for a given algorithm on a single episode

        Args:
            env_name: Name of the gym environment to roll out `algorithm` in, it will be instantiated using gym.make or make_vec_env
            algorithm (PPOActor): Actor whose policy network will be rolled out for the episode. If
            no algorithm is passed in, a random policy will be visualized.
            video_name (str): Name for the mp4 file of the episode that will be saved (omit .mp4). Only used
            when running on local machine.
    """

    def get_action(obs):
        if not algorithm:
            return env.action_space.sample()
        else:
            return algorithm.predict(obs)[0]

    if USING_COLAB:
        from renderlab import RenderFrame
        # visualization probably wont work on colab with my new changes

        directory = './video'
        env_args['render_mode'] = 'rgb_array'
        env = gym.make(env_name, **env_args)
        env = RenderFrame(env, directory)
        obs, info = env.reset()

        while True:
            action = get_action(obs)
            obs, reward, done, truncate, info = env.step(action)

            if done:
                break

        env.play()
    else:
        import cv2
        # need to specify dimensions correctly (use the print statement)
        video = cv2.VideoWriter(f"videos/{video_name}.mp4", cv2.VideoWriter_fourcc(*'mp4v'), 24, (160,210))

        env_args['render_mode'] = 'rgb_array'
        #env = gym.make(env_name, **env_args)
        #env = gym.wrappers.AtariPreprocessing(env)
        env = make_atari_env(env_name, n_envs=1, env_kwargs=env_args)
        env = VecFrameStack(env, n_stack=4)
        #env.metadata['render_fps'] = 60
        obs = env.reset()
        #img = plt.imshow(env.render()) # only call this once
        while True:
            #img.set_data(env.render()) # just update the data
            #display.display(plt.gcf())
            #display.clear_output(wait=True)
            action = get_action(obs)
            if(not algorithm):    
                action = [action]
            obs, reward, done, info  = env.step(action)
            if done:
                print(reward)
                break
            im = env.render()
            im = im[:,:,::-1]
            #print(im.shape)

            video.write(im)

        video.release()
        env.close()
        print(f"Video saved as {video_name}.mp4")

In [7]:
def evaluate_policy(actor, environment, num_episodes=100, progress=True):
    '''
        Returns the mean trajectory reward of rolling out `actor` on `environment

        Parameters
        - actor: PPOActor instance, defined in Part 1
        - environment: classstable_baselines3.common.vec_env.VecEnv instance
        - num_episodes: total number of trajectories to collect and average over
    '''
    total_rew = 0

    iterate = (trange(num_episodes) if progress else range(num_episodes))
    for _ in iterate:
        obs = environment.reset()
        done = False

        while not done:
            action = actor.predict(obs)[0]

            next_obs, reward, done, info = environment.step(action)
            total_rew += reward

            obs = next_obs

    return (total_rew / num_episodes).item()


In [9]:
# for testing if visualization works (random actions selected)
# env arguments are space invader specific
# see https://ale.farama.org/environments/#2 for description of params
# we use frameskip for training but for visualization we don't use frameskip so we can see what's happening clearly
# for dqn we evaluate on same environment as testing
env_id = "ALE/BasicMath-v5"
env_args = {
    'mode':6,
    'difficulty':2,
    'obs_type':"rgb",
    'full_action_space':False,
    'frameskip':1
    }
visualize(env_id, env_args=env_args, video_name="math_test")
# reward is 0 because randomly picking actions gives you no right answers

[0.]
Video saved as math_test.mp4


# Initialization
from A5 starter code

In [10]:
# just one environment for evaluation
env1 = make_atari_env(env_id, n_envs=1, env_kwargs=env_args)
env1 = VecFrameStack(env1, n_stack=4)

In [11]:
# 8 environments
num_envs = 8
env8 = make_atari_env(env_id, n_envs=num_envs, env_kwargs=env_args)
env8 = VecFrameStack(env8, n_stack=4)

In [11]:
eval_callback = EvalCallback(
    eval_env=env1,
    callback_on_new_best=None,
    callback_after_eval=None,
    n_eval_episodes=20,
    eval_freq=200_000,
    log_path='./logs/math_new_env1/',
    best_model_save_path='./models/ppo_model_math_env1/',
    deterministic=True,
    render=False,
    verbose=1,
    warn=True
)

# PPO Model

In [12]:
ppomodel = PPO(
    "CnnPolicy",
    env1,
    learning_rate=2.5e-4,
    n_steps=128,
    batch_size=256,
    n_epochs=4,
    gamma=0.99,
    gae_lambda=0.95,
    clip_range=0.1,
    ent_coef=0.01,
    vf_coef=0.5,
    max_grad_norm=0.5,
)

We recommend using a `batch_size` that is a factor of `n_steps * n_envs`.
Info: (n_steps=128 and n_envs=1)
  warnings.warn(



## Training PPO model

In [13]:
reseed(seed)
#ckpt_path = 'space_invaders'
# If you get the error that Only one live display may be active at once, either restart colab or set progress bar = False
total_timesteps = 10_000_000

ppomodel.learn(total_timesteps=total_timesteps, callback=eval_callback, progress_bar=True)

  warnings.warn("Training and eval env are not of the same type" f"{self.training_env} != {self.eval_env}")



Output()

Eval num_timesteps=200000, episode_reward=0.00 +/- 0.00

Episode length: 2706.65 +/- 14.16

New best mean reward!

...

Episode length: 16522.60 +/- 9.37

Eval num_timesteps=9800000, episode_reward=0.00 +/- 0.00

Episode length: 16446.70 +/- 334.31

Eval num_timesteps=10000000, episode_reward=0.00 +/- 0.00

Episode length: 16523.90 +/- 7.48

In [14]:
best_ppo_model = PPO.load("models/ppo_model_math_env1/best_model.zip", env=env1)

print(evaluate_policy(best_ppo_model, env1))

100%|██████████| 100/100 [02:23<00:00,  1.44s/it]

0.0


In [15]:
visualize(env_id, algorithm=best_ppo_model, video_name='ppo_math_env1', env_args=env_args)

# might need to rerun with best_ppo_model

  logger.warn(



[0.]
Video saved as ppo_math_env1.mp4


In [16]:
ppomodel.save("models/math_env1_ppo_10m_2")

# best model_1 and best_model_2 are using difficulty 1

# DQN Model

Needs to train on a different env with different params due to dqn buffer memory size

In [14]:
from stable_baselines3 import DQN

# can only use env1 instead of env8 for training due to memory constraints

dqnmodel = DQN(
    "CnnPolicy",  # Using CNN policy for visual inputs
    env1,
    learning_rate=1e-4,
    buffer_size=100_000,            # Smaller replay buffer
    learning_starts=50_000,        # Start learning earlier
    batch_size=32,
    train_freq=4,                  # Train more frequently
    target_update_interval=1000,  # Update target network more frequently
    exploration_fraction=0.1,     # Faster exploration decay
    exploration_final_eps=0.01,
    gamma=0.99,
)

In [18]:
eval_callback_dqn = EvalCallback(
    eval_env=env1,
    callback_on_new_best=None,
    callback_after_eval=None,
    n_eval_episodes=10,
    eval_freq=200_000,
    log_path='./logs/new_env_dqn_math/',
    best_model_save_path='./models/new_env_dqn_math/',
    deterministic=True,
    render=False,
    verbose=1,
    warn=True
)

In [19]:
reseed(seed)

# If you get the error that Only one live display may be active at once, either restart colab or set progress bar = False
total_timesteps = 10_000_000
dqnmodel.learn(total_timesteps=total_timesteps, callback=eval_callback_dqn, progress_bar=True)

  warnings.warn("Training and eval env are not of the same type" f"{self.training_env} != {self.eval_env}")



Output()

Eval num_timesteps=200000, episode_reward=0.00 +/- 0.00

Episode length: 15618.30 +/- 2720.44

New best mean reward!

...

Eval num_timesteps=8400000, episode_reward=0.00 +/- 0.00

Episode length: 2779.30 +/- 226.69

Eval num_timesteps=8600000, episode_reward=0.00 +/- 0.00

Episode length: 16391.80 +/- 389.01

KeyboardInterrupt: 

In [ ]:

best_dqn_model = DQN.load("models/new_env_dqn_math/best_model.zip", env=env1)
print(evaluate_policy(best_dqn_model, env1))

  warnings.warn(

100%|██████████| 100/100 [01:00<00:00,  1.66it/s]

8.0600004196167


In [ ]:
visualize('ALE/BasicMath-v5', algorithm=best_dqn_model, video_name='new_dqn_math', env_args=env_args)

[0.]
Video saved as dqn_basic_math.mp4


In [12]:
def eval_policy_2(actor, environment, num_episodes=100, progress=True):
    episode_rewards = []

    iterate = (trange(num_episodes) if progress else range(num_episodes))
    for _ in iterate:
        obs = environment.reset()
        done = False
        total_rew = 0

        while not done:
            action = actor.predict(obs)[0]

            next_obs, reward, done, info = environment.step(action)
            total_rew += reward

            obs = next_obs

        episode_rewards.append(total_rew)

    mean_reward = np.mean(episode_rewards)
    std_reward = np.std(episode_rewards)

    return mean_reward.item(), std_reward.item()

In [15]:
# just evaluate without clipped rewards
model2 = PPO.load("models/best_ppo_model_math_env1/best_model.zip", env=env1)
model3 = DQN.load("models/dqn_basic_math/best_model.zip")
wrapper_args = {"clip_reward":False}
eval_env = make_atari_env(env_id, n_envs=1, env_kwargs=env_args, wrapper_kwargs=wrapper_args)
eval_env = VecFrameStack(eval_env, n_stack=4)
eval_env.seed(42)
print(eval_policy_2(model2, eval_env))

100%|██████████| 100/100 [02:16<00:00,  1.37s/it]

(0.38999998569488525, 0.5982474684715271)


In [16]:
print(eval_policy_2(model3, eval_env))

100%|██████████| 100/100 [01:02<00:00,  1.61it/s]

(8.079999923706055, 0.9019978046417236)


In [ ]:
visualize(env_id, algorithm=model3, video_name='dqn_vid_math', env_args=env_args)


[0.]
Video saved as ppo_vid_math.mp4
[0.]
Video saved as dqn_vid_math.mp4


In [20]:
visualize(env_id, algorithm=model2, video_name='ppo_vid_math', env_args=env_args)


[0.]
Video saved as ppo_vid_math.mp4
